In [1]:
from WNN import WindNeuralNetwork,prepare_new_data

Using TensorFlow backend.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pickle
import paramiko

from datetime import datetime,timedelta
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.triggers.cron import CronTrigger
from IPython.display import clear_output

In [3]:
#PATHS IN THE CLUSTER
c_path='/scratch/wrf/scripts-validation/wrfv4_commands/' # path for the commands
pp_path=c_path+'postprocess/'                            # path for the postprocess
# PATH IN THE COMPUTER
local_path='C:/Users/students/Desktop/wrf_ml/data_from_cluster/'

In [4]:
# duration of the simulation MUST BE INTEGER!
RUN_TIME = 12 

#the forecast is delayed the estimated time that it takes the simulation to be completed 
START_DELAY = int(RUN_TIME / 4) 

In [5]:
with open('wnn_dict_kw_v3.pickle', 'rb') as handle:
    wnn_dict = pickle.load(handle)

C:\Users\students\Anaconda3\envs\wnn\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\students\Anaconda3\envs\wnn\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.3 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [6]:
def gen_commands(star_time):
    dltime=star_time - timedelta(hours=6)
    dlhour=dltime.hour
    dl_str='{:%Y %m %d}'.format(dltime)+' '+'{:02d}'.format(dlhour)
    end_str='{:%Y %m %d %H %h}'.format(star_time + timedelta(hours=RUN_TIME + START_DELAY)).lower()
    start_str='{:%Y %m %d %H}'.format(star_time + timedelta(hours=START_DELAY))
    file_name='WRF_data_'+'{:%Y_%#m_%#d_%#H}'.format(star_time + timedelta(hours=START_DELAY))+'.txt'
    
    commands=[]
    commands.append(c_path+'./WRF-preprocess.sh'+' '+dl_str+' '+start_str+' '+end_str)
    commands.append(c_path+'./WRFprocess2.sh')
    commands.append(c_path+'./postprocess.sh')
    
    return (commands,file_name)

def launch_wrf(star_time=True,verbose=0):
    if star_time==True:
        star_time=datetime.now()
    print('ACTUAL STARTING TIME = {}'.format(star_time))
    
    commands,file_name=gen_commands(star_time=star_time)
    
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    client.connect('enacit-node05.epfl.ch', username='wrf', password='Salut')
    stdin, stdout, stderr=client.exec_command('hostname;w')
    for line in stdout:
        print(line)
       
    #Launc WRF
    for c in commands:
        stdin, stdout, stderr=client.exec_command(c)
        
        if verbose:
            for line in stdout:
                print(line)
                
        for line in stderr:
            print(line)
    #Copy output file
    sftp = client.open_sftp()

    file_remote = pp_path + file_name
    file_local = local_path + file_name

    print(file_remote + '>>>' + file_local)
    sftp.get(file_remote, file_local)
    sftp.close()
    client.close()
    
    return file_local

In [7]:
def wrapper_wrf_prediction_v2():
    clear_output()
    star_time=datetime.now()
    print('ACTUAL STARTING TIME = {}'.format(star_time))
    
    commands,file_name=gen_commands(star_time=star_time)
    
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    client.connect('enacit-node05.epfl.ch', username='wrf', password='Salut')
    stdin, stdout, stderr=client.exec_command('hostname;w')
    for line in stdout:
        print(line)
       
    #Launc WRF
    for c in commands:
        stdin, stdout, stderr=client.exec_command(c)
        print(c)
        print('ACTUAL COMMAND TIME = {}'.format(datetime.now()))
        
        for line in stdout:
            print(line)

        for line in stderr:
            print(line)
    
    exit_status = stdout.channel.recv_exit_status()          # Blocking call
    if exit_status == 0:
        print ("WRF FINISHED - {}".format(datetime.now()))

        
        #Copy output file
        sftp = client.open_sftp()

        file_remote = pp_path + file_name
        file_local = local_path + file_name

        print(file_remote + ' >>> ' + file_local)
        sftp.get(file_remote, file_local)
        sftp.close()
        client.close()

        df1_new,df2_new=prepare_new_data(file_local)

        df_out=wnn_dict['WTG01'].predict_new_data(df1_new['WTG01'],df2_new['WTG01'])
        plt.plot(df_out['Date-time'],df_out['Predicted_Power'])
        # beautify the x-labels
        plt.gcf().autofmt_xdate()
        plt.show()
        
        df_out.to_csv('real_time_predictions/WTG01'+'_'.join([str(star_time.day),str(star_time.month),str(star_time.year),'.csv']))
        
    else:
        client.close()
        print("Error", exit_status)

In [ ]:
print('WAITING FOR THE RIGHT TIME TO START')
sched = BlockingScheduler()
sched.add_job(wrapper_wrf_prediction_v2,CronTrigger.from_crontab('0 */6 * * *'))
sched.start()

ACTUAL STARTING TIME = 2020-08-13 12:00:00.184712
enacit-node05

 10:00:00 up 650 days, 23:44,  0 users,  load average: 0.06, 0.08, 0.02

USER     TTY      FROM             LOGIN@   IDLE   JCPU   PCPU WHAT

/scratch/wrf/scripts-validation/wrfv4_commands/./WRF-preprocess.sh 2020 08 13 06 2020 08 13 15 2020 08 14 03 aug
ACTUAL COMMAND TIME = 2020-08-13 12:00:00.394224
2020-08-13T10:00:00+00:00 WRF cycle launched

2020-08-13T10:00:02+00:00 end of erase.sh

2020-08-13T10:05:27+00:00 end of download.sh

2020-08-13T10:05:27+00:00 end of update.sh

 *** Starting program ungrib.exe ***

Start_date =  2020-08-13_15:00:00 ,      End_date = 2020-08-14_03:00:00

output format is WPS

Path to intermediate files is ./



###############################################################################



Inventory for date = 2020-08-13 15:00:00



PRES   TT       UU       VV       RH       HGT      PSFC     PMSL     SM000010 SM010040 SM040100 SM100200 SM010200 ST000010 ST010040 ST040100 ST100200 ST010



###############################################################################



Inventory for date = 2020-08-13 18:00:00



PRES   TT       UU       VV       RH       HGT      PSFC     PMSL     SM000010 SM010040 SM040100 SM100200 SM010200 ST000010 ST010040 ST040100 ST100200 ST010200 SEAICE   LANDSEA  LANDN    SOILHGT  SKINTEMP SNOW     SNOWH    UMAXW    VMAXW    PMAXW    PMAXWNN  TMAXW    HGTMAXW  UTROP    VTROP    PTROP    PTROPNN  TTROP    HGTTROP 

-------------------------------------------------------------------------------

2013.0  O        O        O        O        O        O        X        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O      

2001.0  X        X        X        X        O        X        O        X        X        X        X        O        X        X



###############################################################################



Inventory for date = 2020-08-13 21:00:00



PRES   TT       UU       VV       RH       HGT      PSFC     PMSL     SM000010 SM010040 SM040100 SM100200 SM010200 ST000010 ST010040 ST040100 ST100200 ST010200 SEAICE   LANDSEA  LANDN    SOILHGT  SKINTEMP SNOW     SNOWH    UMAXW    VMAXW    PMAXW    PMAXWNN  TMAXW    HGTMAXW  UTROP    VTROP    PTROP    PTROPNN  TTROP    HGTTROP 

-------------------------------------------------------------------------------

2013.0  O        O        O        O        O        O        X        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O      

2001.0  X        X        X        X        O        X        O        X        X        X        X        O        X        X



###############################################################################



Inventory for date = 2020-08-14 00:00:00



PRES   TT       UU       VV       RH       HGT      PSFC     PMSL     SM000010 SM010040 SM040100 SM100200 SM010200 ST000010 ST010040 ST040100 ST100200 ST010200 SEAICE   LANDSEA  LANDN    SOILHGT  SKINTEMP SNOW     SNOWH    UMAXW    VMAXW    PMAXW    PMAXWNN  TMAXW    HGTMAXW  UTROP    VTROP    PTROP    PTROPNN  TTROP    HGTTROP 

-------------------------------------------------------------------------------

2013.0  O        O        O        O        O        O        X        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O      

2001.0  X        X        X        X        O        X        O        X        X        X        X        O        X        X



###############################################################################



Inventory for date = 2020-08-14 03:00:00



PRES   TT       UU       VV       RH       HGT      PSFC     PMSL     SM000010 SM010040 SM040100 SM100200 SM010200 ST000010 ST010040 ST040100 ST100200 ST010200 SEAICE   LANDSEA  LANDN    SOILHGT  SKINTEMP SNOW     SNOWH    UMAXW    VMAXW    PMAXW    PMAXWNN  TMAXW    HGTMAXW  UTROP    VTROP    PTROP    PTROPNN  TTROP    HGTTROP 

-------------------------------------------------------------------------------

2013.0  O        O        O        O        O        O        X        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O        O      

2001.0  X        X        X        X        O        X        O        X        X        X        X        O        X        X

...


 87350K .......... .......... .......... .......... .......... 88%  205K 6s

 87400K .......... .......... .......... .......... .......... 88%  111M 6s

 87450K .......... .......... .......... .......... .......... 88% 90.8M 6s

 87500K .......... .......... .......... .......... .......... 88% 23.2M 6s

 87550K .......... .......... .......... .......... .......... 88% 5.06M 6s

 87600K .......... .......... .......... .......... .......... 89%  415K 6s

 87650K .......... .......... .......... .......... .......... 89% 10.4M 6s

 87700K .......... .......... .......... .......... .......... 89% 3.82M 6s

 87750K .......... .......... .......... .......... .......... 89%  438K 6s

 87800K .......... .......... .......... .......... .......... 89% 6.57M 6s

 87850K .......... .......... .......... .......... .......... 89%  496K 6s

 87900K .......... .......... .......... .......... .......... 89% 2.08M 6s

 87950K .......... .......... .......... .......... .......... 89% 3.42M 6s

 79200K .......... .......... .......... .......... .......... 79%  428K 14s

 79250K .......... .......... .......... .......... .......... 79% 7.25M 14s

 79300K .......... .......... .......... .......... .......... 79% 14.4M 14s

 79350K .......... .......... .......... .......... .......... 79% 1.21M 14s

 79400K .......... .......... .......... .......... .......... 80%  577K 14s

 79450K .......... .......... .......... .......... .......... 80% 20.1M 14s

 79500K .......... .......... .......... .......... .......... 80% 6.06M 14s

 79550K .......... .......... .......... .......... .......... 80%  434K 14s

 79600K .......... .......... .......... .......... .......... 80% 8.02M 14s

 79650K .......... .......... .......... .......... .......... 80% 6.76M 14s

 79700K .......... .......... .......... .......... .......... 80%  428K 14s

 79750K .......... .......... .......... .......... .......... 80% 8.05M 14s

 79800K .......... .......... .......... .......... .......... 8

 64600K .......... .......... .......... .......... .......... 65%  109M 2s

 64650K .......... .......... .......... .......... .......... 65% 95.2M 2s

 64700K .......... .......... .......... .......... .......... 65% 98.4M 2s

 64750K .......... .......... .......... .......... .......... 65% 71.8M 2s

 64800K .......... .......... .......... .......... .......... 65% 82.4M 2s

 64850K .......... .......... .......... .......... .......... 66% 77.1M 2s

 64900K .......... .......... .......... .......... .......... 66%  102M 2s

 64950K .......... .......... .......... .......... .......... 66%  108M 2s

 65000K .......... .......... .......... .......... .......... 66% 78.3M 2s

 65050K .......... .......... .......... .......... .......... 66% 72.5M 2s

 65100K .......... .......... .......... .......... .......... 66% 69.2M 2s

 65150K .......... .......... .......... .......... .......... 66% 85.3M 2s

 65200K .......... .......... .......... .......... .......... 66% 66.3M 2s


 61750K .......... .......... .......... .......... .......... 62% 4.02M 30s

 61800K .......... .......... .......... .......... .......... 62% 12.3M 30s

 61850K .......... .......... .......... .......... .......... 62% 1.09M 30s

 61900K .......... .......... .......... .......... .......... 62%  749K 30s

 61950K .......... .......... .......... .......... .......... 62% 27.6M 30s

 62000K .......... .......... .......... .......... .......... 62% 4.34M 30s

 62050K .......... .......... .......... .......... .......... 62% 9.44M 30s

 62100K .......... .......... .......... .......... .......... 62% 22.4M 29s

 62150K .......... .......... .......... .......... .......... 62%  453K 29s

 62200K .......... .......... .......... .......... .......... 62% 26.0M 29s

 62250K .......... .......... .......... .......... .......... 62% 4.78M 29s

 62300K .......... .......... .......... .......... .......... 62% 8.52M 29s

 62350K .......... .......... .......... .......... .......... 6


 51450K .......... .......... .......... .......... .......... 52% 67.2M 5s

 51500K .......... .......... .......... .......... .......... 52% 95.7M 5s

 51550K .......... .......... .......... .......... .......... 52% 77.3M 5s

 51600K .......... .......... .......... .......... .......... 52% 69.6M 5s

 51650K .......... .......... .......... .......... .......... 52%  105M 5s

 51700K .......... .......... .......... .......... .......... 52% 70.5M 5s

 51750K .......... .......... .......... .......... .......... 52% 28.7M 5s

 51800K .......... .......... .......... .......... .......... 52% 17.7M 5s

 51850K .......... .......... .......... .......... .......... 52% 41.7M 5s

 51900K .......... .......... .......... .......... .......... 52% 56.1M 5s

 51950K .......... .......... .......... .......... .......... 52%  490K 5s

 52000K .......... .......... .......... .......... .......... 52% 32.2M 5s

 52050K .......... .......... .......... .......... .......... 53% 54.2M 5s